#### Format the data set

In [1]:
import pandas as pd
from sklearn import preprocessing as prep

In [2]:
data = pd.read_csv('/data/pnc/inbound/clv_lc_201705.csv')

In [3]:
data_cleaned = data.loc[:, ['hh_agr_type', 'new_hh_12m', 'rel_tenure', 'age_hh', 'age_grp_4L', 'inc_code_hh', 'inc_grp_3L', 
                            'consumer_segment', 'psycle_code_ne', 'lifestage', 'dma', 'footprint_clv', 'market_clv',
                            'tch_dmail_impr_lag', 'tch_email_impr_lag', 'cim_olb_impr_lag', 'cim_ccc_impr_lag',
                            'cim_atm_impr_lag', 'cim_tel_impr_lag', 'cim_pli_impr_lag', 'cim_plo_impr_lag', 'class_modal',
                            'dd_hh', 'sv_hh', 'mm_hh', 'cd_hh', 'ir_hh', 'pp_hh', 'cc_hh', 'mt_hh', 'heil_hh', 'heloc_hh', 
                            'pil_hh', 'ploc_hh', 'auto_hh', 'sl_hh', 'iil_hh', 'sdin_hh', 'dc_flag', 'cc_flag', 'od_late_fee']]

In [4]:
data_cleaned = data_cleaned.loc[data_cleaned["class_modal"] == 1]
data_cleaned = data_cleaned.drop('class_modal', axis=1)
data_cleaned.shape

(197358, 40)

In [5]:
data_cleaned = data_cleaned.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [6]:
char_var = ['hh_agr_type', 'age_grp_4L', 'inc_grp_3L', 'consumer_segment', 'lifestage', 'dma', 'market_clv']
for x in char_var:
    coding = prep.LabelEncoder()
    data_cleaned[x] = coding.fit_transform(data_cleaned[x])

In [7]:
labels = data_cleaned.loc[:, 'dd_hh':'od_late_fee']
features = data_cleaned.loc[:, 'hh_agr_type':'cim_plo_impr_lag']

#### Separate train and test data sets

In [8]:
import numpy as np

In [9]:
sample = np.random.rand(len(features))
train = sample <= 0.6
valid = ((sample > 0.6) & (sample <= 0.8))
test = sample > 0.8

In [10]:
features_train = features[train].values
print(features_train.shape)
features_valid = features[valid].values
print(features_valid.shape)
features_test = features[test].values
print(features_test.shape)

(118529, 21)
(39300, 21)
(39529, 21)


In [11]:
labels_train = labels[train].values
print(labels_train.shape)
labels_valid = labels[valid].values
print(labels_valid.shape)
labels_test = labels[test].values
print(labels_test.shape)

(118529, 19)
(39300, 19)
(39529, 19)


#### Naive multi-task learning

##### Keras

In [12]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Dropout
from keras.models import Model

Using TensorFlow backend.


In [13]:
inputs = Input(shape=(21,), dtype='float32')
input_batchnorm = BatchNormalization(momentum=0.9)(inputs)

In [14]:
shared_hidden1 = Dense(100, activation=None)(input_batchnorm)
shared_batchnorm1 = BatchNormalization(momentum=0.9)(shared_hidden1)
shared_activation1 = Activation('elu')(shared_batchnorm1)
shared_dropout1 = Dropout(0.2)(shared_activation1)

In [15]:
shared_hidden2 = Dense(50, activation=None)(shared_dropout1)
shared_batchnorm2 = BatchNormalization(momentum=0.9)(shared_hidden2)
shared_activation2 = Activation('elu')(shared_batchnorm2)
shared_dropout2 = Dropout(0.2)(shared_activation2)

In [16]:
shared_hidden3 = Dense(21, activation=None)(shared_dropout2)
shared_batchnorm3 = BatchNormalization(momentum=0.9)(shared_hidden3)
shared_activation3 = Activation('elu')(shared_batchnorm3)
shared_dropout3 = Dropout(0.2)(shared_activation3)

In [17]:
separated_hidden1 = [Dense(100, activation=None)(shared_dropout3) for k in range(19)]
separated_batchnorm1 = [BatchNormalization(momentum=0.9)(separated_hidden1[k]) for k in range(19)]
separated_activation1 = [Activation('elu')(separated_batchnorm1[k]) for k in range(19)]
separated_dropout1 = [Dropout(0.2)(separated_activation1[k]) for k in range(19)] 

In [18]:
separated_hidden2 = [Dense(50, activation=None)(separated_dropout1[k]) for k in range(19)]
separated_batchnorm2 = [BatchNormalization(momentum=0.9)(separated_hidden2[k]) for k in range(19)]
separated_activation2 = [Activation('elu')(separated_batchnorm2[k]) for k in range(19)]
separated_dropout2 = [Dropout(0.2)(separated_activation2[k]) for k in range(19)]

In [19]:
outputs = [Dense(1, activation=None)(separated_dropout2[k]) for k in range(19)]
output_batchnorm = [BatchNormalization(momentum=0.9)(outputs[k]) for k in range(19)]
output_activation = [Activation('softmax')(output_batchnorm[k]) for k in range(19)]

In [20]:
model = Model(inputs=inputs, outputs=outputs)

In [21]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(features_train, [labels_train[:,k] for k in range(19)], batch_size=1000, epochs=30, 
          validation_data=(features_valid, [labels_valid[:,k] for k in range(19)]))

In [ ]:
model.evaluate(features_test, [labels_test[:,k] for k in range(19)], batch_size=1000)

##### tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
inputs = tf.placeholder(tf.float64, shape=[None, 21])

In [ ]:
hidden1 = tf.contrib.layers.fully_connected(inputs, 52, activation_fn=None)
norm1 = tf.contrib.layers.batch_norm(hidden1, center=True, scale=True)
out1 = tf.nn.relu(norm1)

In [ ]:
hidden2 = tf.contrib.layers.fully_connected(out1, 10, activation_fn=None)
norm2 = tf.contrib.layers.batch_norm(hidden2, center=True, scale=True)
out2 = tf.nn.relu(norm2)

In [ ]:
outputs = [tf.contrib.layers.fully_connected(out2, 1) for i in range(19)]

In [ ]:
labels = [tf.placeholder(tf.float64, shape=[None, 1]) for i in range(19)]

In [ ]:
losses = [tf.nn.softmax_cross_entropy_with_logits(logits=outputs[i], labels=labels[i]) for i in range(19)]

In [ ]:
joint_loss = sum(losses)

In [ ]:
optimizers = [tf.train.AdamOptimizer(0.01).minimize(losses[i]) for i in range(19)]

In [ ]:
joint_optimizer = tf.train.AdamOptimizer(0.01).minimize(joint_loss)